In [11]:
# packages gerais
import pandas as pd
import joblib

# df_sales_ dataset
df_sales_filtered_all = joblib.load('df_sales_filtered_all.pkl')
df_sales_filtered_2018_2019 = joblib.load('df_sales_filtered_2018_2019.pkl')
df_sales_filtered_2019 = joblib.load('df_sales_filtered_2019.pkl')
df_sales_filtered_last_6_month = joblib.load('df_sales_filtered_last_6_month.pkl')

In [12]:
df_sales_filtered_last_6_month

,store_id,product_id,year,week,sales,revenue,stock,price
0,S0020,P0001,2019,14,2.0,18.56,9.0,10.95
1,S0020,P0001,2019,15,1.0,9.28,8.0,10.95
2,S0020,P0001,2019,16,1.0,9.28,7.0,10.95
3,S0020,P0001,2019,17,2.0,18.56,5.0,10.95
4,S0020,P0001,2019,18,2.0,18.56,3.0,10.95
...,...,...,...,...,...,...,...,...
41889,S0097,P0748,2019,36,2.0,28.00,10.0,18.90
41890,S0097,P0748,2019,37,0.0,0.00,10.0,18.90
41891,S0097,P0748,2019,38,1.0,17.50,9.0,18.90
41892,S0097,P0748,2019,39,1.0,17.50,8.0,18.90
